In [1]:
!pip install -q ultralytics numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
from pathlib import Path
from collections import deque
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#### **Drowsiness Detector Class:**

In [3]:
class DrowsinessDetector:
    def __init__(self, fps=30):
        self.fps = fps
        self.eye_closed_frames = 0
        self.yawn_frames = 0
        self.alert_cooldown = 0

        self.eye_threshold = int(0.7 * fps)    # 0.7 seconds
        self.yawn_threshold = int(0.5 * fps)   # 0.5 seconds
        self.cooldown_duration = int(3.0 * fps)

        self.total_alerts = {"CRITICAL": 0, "WARNING": 0}
        self.frame_history = deque(maxlen=int(30 * fps))

    def update(self, eyes_closed, eyes_open, yawning):
        # Eye closure
        if eyes_closed:
            self.eye_closed_frames += 1
        elif eyes_open:
            self.eye_closed_frames = max(0, self.eye_closed_frames - 3)
        else:
            self.eye_closed_frames = max(0, self.eye_closed_frames - 1)

        # Yawning
        if yawning:
            self.yawn_frames += 1
        else:
            self.yawn_frames = max(0, self.yawn_frames - 2)

        # Cooldown
        if self.alert_cooldown > 0:
            self.alert_cooldown -= 1

        # Determine alert
        alert_level = "NONE"
        if self.alert_cooldown == 0:
            if self.eye_closed_frames > self.eye_threshold:
                alert_level = "CRITICAL"
                self.alert_cooldown = self.cooldown_duration
                self.total_alerts["CRITICAL"] += 1
            elif self.yawn_frames > self.yawn_threshold:
                alert_level = "WARNING"
                self.alert_cooldown = self.cooldown_duration // 2
                self.total_alerts["WARNING"] += 1

        self.frame_history.append({
            'eyes_closed': eyes_closed,
            'yawning': yawning,
            'alert': alert_level
        })

        return alert_level

    def get_stats(self):
        return {
            "eye_closed_frames": self.eye_closed_frames,
            "yawn_frames": self.yawn_frames,
            "total_alerts": sum(self.total_alerts.values()),
            "critical_alerts": self.total_alerts["CRITICAL"],
            "warning_alerts": self.total_alerts["WARNING"]
        }


#### **Overlay Drawing:**

In [4]:
def draw_overlay(frame, results, alert_level, detector, fps_display, model_names):
    h, w = frame.shape[:2]

    # Draw detection boxes
    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        colors = {0: (0,0,255), 1: (0,255,0), 2: (0,165,255)}
        color = colors.get(cls, (255,255,255))

        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        label = f"{model_names[cls]} {conf:.2f}"
        size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        cv2.rectangle(frame, (x1, y1-size[1]-10), (x1+size[0], y1), color, -1)
        cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

    # Alert banners
    if alert_level == "CRITICAL":
        overlay = frame.copy()
        cv2.rectangle(overlay, (0,0), (w,150), (0,0,200), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        pulse = int(abs(np.sin(time.time()*5))*50 + 205)
        cv2.putText(frame, "!!! DROWSINESS ALERT !!!", (w//2-300,100),
                    cv2.FONT_HERSHEY_DUPLEX, 2.0, (pulse,pulse,255), 5, cv2.LINE_AA)
    elif alert_level == "WARNING":
        overlay = frame.copy()
        cv2.rectangle(overlay, (0,0), (w,120), (0,140,255), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        cv2.putText(frame, "WARNING: Fatigue Detected", (w//2-250,80),
                    cv2.FONT_HERSHEY_DUPLEX, 1.5, (255,255,255), 4, cv2.LINE_AA)

    # Stats panel
    stats = detector.get_stats()
    panel_h = 140
    overlay = frame.copy()
    cv2.rectangle(overlay, (10,h-panel_h-10), (350,h-10), (0,0,0), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
    y = h-panel_h+10
    cv2.putText(frame, f"Eye Closure: {stats['eye_closed_frames']} frames", (20,y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    y += 25
    cv2.putText(frame, f"Yawn: {stats['yawn_frames']} frames", (20,y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    y += 25
    cv2.putText(frame, f"Alerts: {stats['total_alerts']} (C:{stats['critical_alerts']} W:{stats['warning_alerts']})",
                (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    y += 30
    cv2.putText(frame, f"FPS: {fps_display:.1f}", (20,y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100,255,100), 1)

    return frame

#### **Inference on Video:**

In [5]:
def test_on_video(model_path, video_path, output_path=None, skip_frames=2):
    model = YOLO(model_path)
    model.fuse()
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if output_path is None:
        output_path = str(Path(video_path).stem + "_output.mp4")
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width,height))

    detector = DrowsinessDetector(fps=fps)
    last_results = None
    frame_count = 0
    processing_times = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        start = time.time()

        if frame_count % skip_frames == 0 or last_results is None:
            results = model.predict(frame, conf=0.4, iou=0.5, max_det=3, verbose=False)[0]
            last_results = results
        else:
            results = last_results

        eyes_closed = any(int(box.cls[0])==0 for box in results.boxes)
        eyes_open = any(int(box.cls[0])==1 for box in results.boxes)
        yawning = any(int(box.cls[0])==2 for box in results.boxes)

        alert_level = detector.update(eyes_closed, eyes_open, yawning)
        processing_time = time.time() - start
        fps_display = 1.0 / processing_time if processing_time>0 else 0
        processing_times.append(processing_time)

        frame = draw_overlay(frame, results, alert_level, detector, fps_display, model.names)
        out.write(frame)

    cap.release()
    out.release()

    avg_fps = 1.0/np.mean(processing_times) if processing_times else 0
    return {"frames_processed": frame_count, "avg_fps": avg_fps, "output_path": output_path,
            "stats": detector.get_stats()}


In [6]:
MODEL_PATH = '/kaggle/input/drowsiness-detector/pytorch/default/1/best.pt'
VIDEO_PATH = '/kaggle/input/testing-video/test_video.mp4'
OUTPUT_PATH = '/kaggle/working/output_video.mp4'

results = test_on_video(MODEL_PATH, VIDEO_PATH, OUTPUT_PATH, skip_frames=2)
results

YOLOv5n summary (fused): 84 layers, 2,503,529 parameters, 0 gradients, 7.1 GFLOPs


{'frames_processed': 617,
 'avg_fps': 137.16530321259268,
 'output_path': '/kaggle/working/output_video.mp4',
 'stats': {'eye_closed_frames': 65,
  'yawn_frames': 0,
  'total_alerts': 7,
  'critical_alerts': 7,
  'warning_alerts': 0}}